# Loading the database

In [1]:
import os
import json
import numpy as np
from mutagen.mp4 import MP4
from datetime import datetime
from bson import ObjectId
from pymongo import MongoClient
from pprint import PrettyPrinter
pp = PrettyPrinter()

In [2]:
MUSIC_DIR = "/Users/elizabethchen/Documents/iPod Library - updated 2025-07/iPod/iPod/Media.localized/Music"

In [3]:
MUSIC_DIR

'/Users/elizabethchen/Documents/iPod Library - updated 2025-07/iPod/iPod/Media.localized/Music'

In [4]:
def get_codec(mf_info_codec):
    if mf_info_codec == 'mp4a.40.2':
        return 'aac'
    elif mf_info_codec == 'alac':
        return 'alac'

In [5]:
for ARTIST_DIR in os.listdir(MUSIC_DIR):
    if not ARTIST_DIR.startswith("."):
        ARTIST_PATH = os.path.join(MUSIC_DIR, ARTIST_DIR)
        for ALBUM_DIR in os.listdir(ARTIST_PATH):
            if not ALBUM_DIR.startswith("."):
                if not ALBUM_DIR.endswith(".jpg"):
                    ALBUM_PATH = os.path.join(ARTIST_PATH, ALBUM_DIR)
                    for SONG in os.listdir(ALBUM_PATH):
                        if not SONG.startswith("."):
                            if not SONG .endswith(".jpg"):
                                try:
                                    SONG_PATH = os.path.join(ALBUM_PATH, SONG)
                                    mf = MP4(SONG_PATH)
                                except: 
                                    pass
                                #title = mf['©nam'][0]
                                #title, artist, album, = mf['©nam'][0], mf['©ART'][0], mf['©alb'][0]
                                #album_artist, release_date = mf['aART'][0], mf['©day'][0]
                                #track_num = mf['trkn'][0][0]
                                #print(title, track_num)
                                #duration, bitrate, codec = mf.info.length, mf.info.bitrate, mf.info.codec
                                

# MongoDB

In [12]:
connection = MongoClient("mongodb://127.0.0.1:27017/museplayer" )
db = connection.museplayer

In [13]:
db.albums

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'museplayer'), 'albums')

### Artists Collection Structure

```
Artists: {
            _id: ObjectID
            artistName: string
            albums: [ list of album_ids ]
            profilePic: [image] or [filename/location]
         }
```

### Albums Collection Structure

```
Albums: { 
           _id: ObjectId
           albumName: string
           albumArtist: string
           releaseDate: date
           albumCover: filename/location
           genre: string
           songs: [ObjectId]
```

In [14]:
albums = db.albums
albums.drop()

artists = db.artists
artists.drop()

songs = db.songs
songs.drop()

for ARTIST_DIR in os.listdir(MUSIC_DIR):
    try:
        if not ARTIST_DIR.startswith("."):
            artist_id = ObjectId()
            artist_album_ids = []
            for ALBUM_DIR in os.listdir(os.path.join(MUSIC_DIR, ARTIST_DIR)):
                # Two files to exclude when searching the artist directory
                if not ALBUM_DIR in {'profile.jpg', '.DS_Store'}: 
                    cover_loc = os.path.join(MUSIC_DIR, ARTIST_DIR, ALBUM_DIR, 'cover.jpg')
                    assert os.path.exists(cover_loc) == True, 'cover does not exist!'
                    album_id =  ObjectId()
                    songs_list = []
                    for SONG in os.listdir(os.path.join(MUSIC_DIR, ARTIST_DIR, ALBUM_DIR)):
                        # Two files to exclude when searching the albums directory
                        if not SONG in {'cover.jpg', '.DS_Store'}: 
                            try:
                                file_loc = os.path.join(MUSIC_DIR, ARTIST_DIR, ALBUM_DIR, SONG)
                                mf = MP4(file_loc)
                                song_entry = {
                                    '_id': ObjectId(),
                                    'artistId': artist_id,
                                    'albumId': album_id,
                                    'songFileLoc': file_loc,
                                    'albumArtworkLoc': cover_loc,
                                    'title': mf['©nam'][0],
                                    'duration': np.round(mf.info.length, 1),
                                    'trackNumber': mf['trkn'][0][0],
                                    'artistName': mf['©ART'][0],
                                    'albumName': mf['©alb'][0],
                                    'albumArtist': mf['aART'][0],
                                    'releaseDate': mf['©day'][0],
                                    'bitrate': np.round(mf.info.bitrate / 1000), 
                                    'codec': get_codec(mf.info.codec),
                                    'isFavorited': False,
                                    'numPlays': 0, 
                                    'lastPlayed': None
                                }
                                songs_list.append(song_entry["_id"])
                                songs.insert_one(song_entry)
                            except:
                                #print(os.path.join(MUSIC_DIR, ARTIST_DIR, ALBUM_DIR, SONG))
                                continue
                    artist_album_ids.append(album_id)
                    album_entry  = {
                        '_id': album_id,
                        'artistId': artist_id,
                        'albumName': mf['©alb'][0],
                        'albumArtist': mf['aART'][0],
                        'releaseDate': mf['©day'][0],
                        'genre': mf['©gen'][0],
                        'songs': songs_list,
                        'albumArtworkLoc': cover_loc,
                        'isFavorited': False
                    }
                    albums.insert_one(album_entry)   
            profile_loc = os.path.join(MUSIC_DIR, ARTIST_DIR, 'profile.jpg')
            assert os.path.exists(profile_loc) == True
            artist_entry = {
                '_id': artist_id,
                'artistName': mf['aART'][0],
                'artistAlbums': artist_album_ids,
                'profilePicLoc': profile_loc,
                'isFavorited': False
            }
            artists.insert_one(artist_entry)
    except:
        continue

In [15]:
artists.create_index('artistName')
songs.create_index('title')
albums.create_index('albumName')

'albumName_1'